In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os
from pathlib import Path
import xarray as xr
import rioxarray as rxr
from affine import Affine
from shapely.geometry import shape
from rasterio.features import shapes
import itertools
from shapely import Polygon, geometry

fe_output_dir = Path(r"p:\11207608-coclico\FULLTRACK_DATA\WP4\front_end_data")

cfhp_stats = gpd.read_parquet(r"p:\11207608-coclico\FULLTRACK_DATA\WP4\LAU_stats\LAU_NUTS_CFHP_31jan_all.parquet")

cfhp_stats

C:\Users\soest\AppData\Local\Temp\ipykernel_15916\402373938.py:1: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


,GISCO_ID,CNTR_CODE,nuts_2,nuts_3,LAU_ID,LAU_NAME,POP_2020,POP_DENS_2,AREA_KM2,YEAR,...,UNDEFENDED_MAPS\1000\High_End\2050\n_less05,UNDEFENDED_MAPS\1000\High_End\2050\n_more05,UNDEFENDED_MAPS\1000\High_End\2050\total,UNDEFENDED_MAPS\1000\High_End\2050\flooded,UNDEFENDED_MAPS\1000\High_End\2100\n_nans,UNDEFENDED_MAPS\1000\High_End\2100\n_no_flood,UNDEFENDED_MAPS\1000\High_End\2100\n_less05,UNDEFENDED_MAPS\1000\High_End\2100\n_more05,UNDEFENDED_MAPS\1000\High_End\2100\total,UNDEFENDED_MAPS\1000\High_End\2100\flooded
1192,DE_03361001,DE,DE93,DE93B,03361001,"Achim, Stadt",31923.0,471.398771,67.719735,2020,...,6.0,0.0,108467.0,0.000055,93163.0,108462.0,5.0,0.0,108467.0,0.000046
1193,DE_03361002,DE,DE93,DE93B,03361002,Blender,2885.0,75.034640,38.448908,2020,...,4.0,0.0,61341.0,0.000065,47469.0,61337.0,4.0,0.0,61341.0,0.000065
1195,DE_03361003,DE,DE93,DE93B,03361003,Dörverden,9009.0,107.940908,83.462333,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1196,DE_03361004,DE,DE93,DE93B,03361004,Emtinghausen,1464.0,68.573846,21.349247,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1197,DE_03361005,DE,DE93,DE93B,03361005,Kirchlinteln,9911.0,56.675823,174.871744,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97667,FR_97614,FR,FRY5,FRY50,97614,Ouangani,10203.0,555.838786,18.356042,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97669,FR_97616,FR,FRY5,FRY50,97616,Sada,11156.0,1012.643086,11.016715,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97670,FR_97617,FR,FRY5,FRY50,97617,Tsingoni,13934.0,406.428213,34.284037,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97660,FR_97608,FR,FRY5,FRY50,97608,Dzaoudzi,17831.0,2797.974566,6.372824,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Print all column names containing 'MAPS'
map_cols = [col for col in cfhp_stats.columns if 'MAPS' in col]
for col in map_cols:
    print(col)

HIGH_DEFENDED_MAPS\static\None\2010\n_nans
HIGH_DEFENDED_MAPS\static\None\2010\n_no_flood
HIGH_DEFENDED_MAPS\static\None\2010\n_less05
HIGH_DEFENDED_MAPS\static\None\2010\n_more05
HIGH_DEFENDED_MAPS\static\None\2010\total
HIGH_DEFENDED_MAPS\static\None\2010\flooded
HIGH_DEFENDED_MAPS\static\None\2030\n_nans
HIGH_DEFENDED_MAPS\static\None\2030\n_no_flood
HIGH_DEFENDED_MAPS\static\None\2030\n_less05
HIGH_DEFENDED_MAPS\static\None\2030\n_more05
HIGH_DEFENDED_MAPS\static\None\2030\total
HIGH_DEFENDED_MAPS\static\None\2030\flooded
HIGH_DEFENDED_MAPS\static\None\2050\n_nans
HIGH_DEFENDED_MAPS\static\None\2050\n_no_flood
HIGH_DEFENDED_MAPS\static\None\2050\n_less05
HIGH_DEFENDED_MAPS\static\None\2050\n_more05
HIGH_DEFENDED_MAPS\static\None\2050\total
HIGH_DEFENDED_MAPS\static\None\2050\flooded
HIGH_DEFENDED_MAPS\static\None\2100\n_nans
HIGH_DEFENDED_MAPS\static\None\2100\n_no_flood
HIGH_DEFENDED_MAPS\static\None\2100\n_less05
HIGH_DEFENDED_MAPS\static\None\2100\n_more05
HIGH_DEFENDED_MAPS\sta

In [ ]:
## Store the data in a new parquet file
cfhp_stats.to_parquet(fe_output_dir.joinpath("cfhp_all_stats.parquet"))

In [ ]:
# For the front-end the there is a maximum amount of rows (255) that is allowed. For that reason we rework the dataset.
# The variables map_type, return_period, scenario and time will be added in a column. Strechting the dataset in the vertical direction.
df = cfhp_stats.copy()

# Identify base columns that remain unchanged
base_columns = [
    "GISCO_ID", "CNTR_CODE", "nuts_2", "nuts_3", "LAU_ID", "LAU_NAME", 
    "POP_2020", "POP_DENS_2", "AREA_KM2", "YEAR", "FID", "geometry"
]

# Identify dynamic columns (all non-base columns)
variable_columns = [col for col in df.columns if col not in base_columns]

# Extract components from column names
df_extracted = pd.DataFrame([col.split('\\') for col in variable_columns], 
                            columns=["map_type", "return_period", "scenario", "time", "impact_type"])
df_extracted["original_column"] = variable_columns  # Keep track of original names

# Melt dataframe to long format
df_long = df.melt(id_vars=base_columns, value_vars=variable_columns, var_name="original_column", value_name="value")

# Merge extracted components directly into the long dataframe
df_long = df_long.merge(df_extracted, on="original_column").drop(columns=["original_column"])

# Pivot to create separate columns for impact types
df_wide = df_long.pivot(index=base_columns + ["map_type", "return_period", "scenario", "time"], 
                        columns="impact_type", 
                        values="value").reset_index()

# Remove column index name
df_wide.columns.name = None


cfhp_stats_gpkg = df_wide.copy()

In [ ]:
# Convert to geodataframe
cfhp_stats_gpkg = gpd.GeoDataFrame(cfhp_stats_gpkg, geometry="geometry")

# Rename FID to LAU_FID
cfhp_stats_gpkg.rename(columns={'FID': 'LAU_FID'}, inplace=True)

# Drop all rows with time == 2150 or scenario == 'High_End'
cfhp_stats_gpkg = cfhp_stats_gpkg[(cfhp_stats_gpkg.time != '2150')]

cfhp_stats_gpkg

,GISCO_ID,CNTR_CODE,nuts_2,nuts_3,LAU_ID,LAU_NAME,POP_2020,POP_DENS_2,AREA_KM2,YEAR,...,map_type,return_period,scenario,time,flooded,n_less05,n_more05,n_nans,n_no_flood,total
0,AL_AL121,AL,AL01,AL012,AL121,Durrës,175110.0,517.362360,338.466834,2019,...,HIGH_DEFENDED_MAPS,1,High_End,2010,0.177121,68679.0,28178.0,577664.0,449985.0,546842.0
1,AL_AL121,AL,AL01,AL012,AL121,Durrës,175110.0,517.362360,338.466834,2019,...,HIGH_DEFENDED_MAPS,1,High_End,2030,0.180424,65065.0,33608.0,577664.0,448222.0,546895.0
2,AL_AL121,AL,AL01,AL012,AL121,Durrës,175110.0,517.362360,338.466834,2019,...,HIGH_DEFENDED_MAPS,1,High_End,2050,0.190083,39181.0,64726.0,577664.0,442732.0,546639.0
3,AL_AL121,AL,AL01,AL012,AL121,Durrës,175110.0,517.362360,338.466834,2019,...,HIGH_DEFENDED_MAPS,1,High_End,2100,0.262857,10660.0,133112.0,577664.0,403187.0,546959.0
4,AL_AL121,AL,AL01,AL012,AL121,Durrës,175110.0,517.362360,338.466834,2019,...,HIGH_DEFENDED_MAPS,1,None,2010,0.177121,68679.0,28178.0,577664.0,449985.0,546842.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2871595,UK_W06000023,UK,UKL2,UKL24,W06000023,Powys,132447.0,25.479361,5198.207327,2020,...,UNDEFENDED_MAPS,static,SSP245,2100,0.000021,98.0,79.0,6420974.0,8314588.0,8314765.0
2871596,UK_W06000023,UK,UKL2,UKL24,W06000023,Powys,132447.0,25.479361,5198.207327,2020,...,UNDEFENDED_MAPS,static,SSP585,2010,0.000004,31.0,6.0,6420974.0,8314728.0,8314765.0
2871597,UK_W06000023,UK,UKL2,UKL24,W06000023,Powys,132447.0,25.479361,5198.207327,2020,...,UNDEFENDED_MAPS,static,SSP585,2030,0.000005,30.0,9.0,6420974.0,8314726.0,8314765.0
2871598,UK_W06000023,UK,UKL2,UKL24,W06000023,Powys,132447.0,25.479361,5198.207327,2020,...,UNDEFENDED_MAPS,static,SSP585,2050,0.000007,44.0,15.0,6420974.0,8314706.0,8314765.0


In [6]:
2871600 / 3 / 4 / 5 / 4 

11965.0

In [ ]:
# Write to geopackage
cfhp_stats_gpkg.to_file(fe_output_dir.joinpath("cfhp_all_stats.gpkg"), driver="GPKG")

In [24]:
# Load the geopackage
cfhp_stats_parq = gpd.read_parquet(fe_output_dir.joinpath("cfhp_all_stats.parquet"))

In [25]:
# List all columns that contain "2150"
cols_2150 = [col for col in cfhp_stats_parq.columns if "2150" in col]
for col in cols_2150:
    print(col)

In [3]:
# Only get unique map_columns
maps_columns = list(set([str(Path(col).parent) for col in cfhp_stats.columns if "MAPS" in col]))

for map_column in maps_columns:
    
    print(map_column)
    # Select all column names containing the current map column
    cur_stats = cfhp_stats.loc[:, cfhp_stats.columns.str.startswith(map_column)]

    # split column names at backslashes, only keep last element
    cur_stats.columns = cur_stats.columns.str.split("\\").str[-1]

    # Add the cur stats to the first 11 columns of pp_stats
    pp_gpkg = pd.concat([cfhp_stats.iloc[:, :12], cur_stats], axis=1)

    # Set FID to LAU_FID for storing as geopackage
    pp_gpkg.rename(columns={'FID': 'LAU_FID'}, inplace=True)

    # Set geopackage file name
    scen_name = map_column.replace("\\", "_")

    vars = map_column.split("\\")
    map_type = vars[0]
    rp = vars[1]
    scen = vars[2]
    year = vars[3]

    gpkg_fp = fe_output_dir.joinpath("map_stats", map_type,rp,scen)
    gpkg_fp.mkdir(parents=True, exist_ok=True)

    gpkg_file = gpkg_fp.joinpath(f"cfhp_all_stats_{scen_name}.gpkg")

    # Write to geopackage
    pp_gpkg.to_file(str(gpkg_file), layer=f'cfhp_all_stats_{scen_name}', driver='GPKG')

    # Write to parquet
    pp_gpkg.to_parquet(gpkg_fp.joinpath(f"cfhp_all_stats_{scen_name}.parquet"))

HIGH_DEFENDED_MAPS\1\High_End\2010
LOW_DEFENDED_MAPS\static\High_End\2100
LOW_DEFENDED_MAPS\1000\None\2030
LOW_DEFENDED_MAPS\1\SSP245\2050
HIGH_DEFENDED_MAPS\static\High_End\2030
UNDEFENDED_MAPS\100\None\2100
UNDEFENDED_MAPS\1\SSP245\2030
LOW_DEFENDED_MAPS\1000\High_End\2010
UNDEFENDED_MAPS\100\SSP585\2100
UNDEFENDED_MAPS\100\High_End\2050
UNDEFENDED_MAPS\1000\SSP245\2050
UNDEFENDED_MAPS\1000\SSP585\2030
HIGH_DEFENDED_MAPS\static\SSP245\2030
HIGH_DEFENDED_MAPS\1\SSP585\2030
UNDEFENDED_MAPS\static\SSP126\2050
LOW_DEFENDED_MAPS\1\High_End\2050
UNDEFENDED_MAPS\1\None\2030
LOW_DEFENDED_MAPS\100\SSP126\2100
HIGH_DEFENDED_MAPS\1\SSP126\2050
HIGH_DEFENDED_MAPS\1000\High_End\2050
HIGH_DEFENDED_MAPS\100\None\2010
HIGH_DEFENDED_MAPS\100\High_End\2050
UNDEFENDED_MAPS\1000\High_End\2030
HIGH_DEFENDED_MAPS\100\SSP245\2100
HIGH_DEFENDED_MAPS\1\SSP245\2010
LOW_DEFENDED_MAPS\100\SSP585\2050
LOW_DEFENDED_MAPS\static\SSP245\2050
UNDEFENDED_MAPS\1\SSP126\2100
UNDEFENDED_MAPS\1\High_End\2010
HIGH_DEFENDED

In [6]:
# for col in RP1[RP1.columns[-5:]]:
#     LAU_NUTS[col] = RP1[col]

# LAU_NUTS[RP1.columns[-5:]]


In [7]:
# remove the LAU's with no data at all.. (quick fix but not very nice, would be better to mask with coastal mask, see this later on)
LAU_NUTS_CLEAN = LAU_NUTS.dropna(thresh=13)

In [8]:
LAU_NUTS.shape

(98613, 192)

In [9]:
LAU_NUTS_CLEAN.shape

(6549, 192)

In [10]:
#if a column contains '\' character, it will be replaced with '_' character
LAU_NUTS_CLEAN.columns = LAU_NUTS_CLEAN.columns.str.replace('\\', '_')

#rename columns with name FID to FID_LAU
LAU_NUTS_CLEAN.rename(columns={'FID': 'FID_LAU'}, inplace=True)

#Create a new column that contains the index of the row
LAU_NUTS_CLEAN['fid'] = LAU_NUTS_CLEAN.index

C:\Users\kras\AppData\Local\Temp\ipykernel_11464\3548292644.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LAU_NUTS_CLEAN.rename(columns={'FID': 'FID_LAU'}, inplace=True)
c:\Users\kras\AppData\Local\mambaforge\envs\coclico\Lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [18]:
#saving
LAU_NUTS_CLEAN.to_parquet(r"p:\11207608-coclico\FULLTRACK_DATA\WP4\LAU_stats\LAU_NUTS_CFHP_CLEAN.parquet")

# Save the GeoDataFrame to a geopackage file
LAU_NUTS_CLEAN.to_file(r'p:\11207608-coclico\FULLTRACK_DATA\WP4\LAU_stats\lau_nuts_cfhp_clean.gpkg', layer='lau_nuts_cfhp_clean', driver='GPKG')
""" gdf.to_parquet('lau_nuts_cfhp.parquet') """

" gdf.to_parquet('lau_nuts_cfhp.parquet') "

In [11]:
# import pandas as pd
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     print(LAU_NUTS.loc[1])

In [12]:
# masking the data with the Coastal Mask (this is a better way to mask the data than removing the LAU's with no data at all)

In [11]:
# masking the data with the Coastal Mask
import pystac_client
catalog = pystac_client.Client.open(
    "https://storage.googleapis.com/coclico-data-public/coclico/coclico-stac-27aug/catalog.json"
)
collection = catalog.get_child(id = 'coastal-mask')

c:\Users\kras\AppData\Local\mambaforge\envs\coclico\Lib\site-packages\pystac_client\client.py:187: NoConformsTo: Server does not advertise any conformance classes.
  warnings.warn(NoConformsTo())


In [13]:
# masking the data with the Coastal Mask (takes abou 45 min)
folder_mask = r"p:\11207608-coclico\FASTTRACK_DATA\19_coastal_mask\cogs"
pgnzd = []
# read tif files in the coastal mask folder
for idx, file in enumerate(os.listdir(folder_mask)):
    print(idx)
    if file.endswith(".tif"):
        img = xr.open_dataset(os.path.join(folder_mask, file), engine="rasterio", mask_and_scale=False) 
        if geometry.box(*img.rio.bounds()).intersects(geometry.box(*totalbbox)): # only do the ones in the LAU_NUTS bbox --> speed up to 17 min
            print(os.path.join(folder_mask, file))

            # get the data
            data_array = img.band_data.values
            mask = data_array == 1
            
            # Split the string into a list of values and correct the gdal affine transformation
            geotrans = list(map(float, img.spatial_ref.GeoTransform.split(' ')))
            afn = Affine.from_gdal(*geotrans)

            # Extract shapes
            results = (
                {'properties': {'value': v}, 'geometry': s}
                for i, (s, v) in enumerate(
                    shapes(data_array, mask=mask, transform=afn))
            )

            pgnzd.append(list(results))

0
1
2
3
4
5
6
p:\11207608-coclico\FASTTRACK_DATA\19_coastal_mask\cogs\B01_x-63.95_y-3.75.tif
7
8
9
10
11
p:\11207608-coclico\FASTTRACK_DATA\19_coastal_mask\cogs\B01_x-26.4_y13.32.tif
12
13
14
15
16
17
18
p:\11207608-coclico\FASTTRACK_DATA\19_coastal_mask\cogs\B01_x-5.92_y-20.81.tif
19
20
21
p:\11207608-coclico\FASTTRACK_DATA\19_coastal_mask\cogs\B01_x-33.23_y33.8.tif
22
23
24
25
p:\11207608-coclico\FASTTRACK_DATA\19_coastal_mask\cogs\B01_x-60.53_y40.63.tif
26
p:\11207608-coclico\FASTTRACK_DATA\19_coastal_mask\cogs\B01_x-9.33_y61.11.tif
27
28
29
30
p:\11207608-coclico\FASTTRACK_DATA\19_coastal_mask\cogs\B01_x41.87_y44.04.tif
31
p:\11207608-coclico\FASTTRACK_DATA\19_coastal_mask\cogs\B01_x48.69_y33.8.tif
32
33
34
35
36
37
38
39
p:\11207608-coclico\FASTTRACK_DATA\19_coastal_mask\cogs\B01_x-16.16_y-3.75.tif
40
41
42
p:\11207608-coclico\FASTTRACK_DATA\19_coastal_mask\cogs\B01_x-40.05_y40.63.tif
43
44
45
46
47
48
p:\11207608-coclico\FASTTRACK_DATA\19_coastal_mask\cogs\B01_x38.45_y44.04.tif
4

In [19]:
len(pgnzd)

1008

In [20]:
# flatten the list
fpgnzd = list(itertools.chain(*pgnzd))

In [21]:
len(fpgnzd)

167957

In [29]:
# Convert shapes to a GeoDataFrame
shapes_gdf = gpd.GeoDataFrame.from_features(fpgnzd)

,geometry,value
0,"POLYGON ((-62.43292 -0.75292, -62.43292 -0.754...",1.0
1,"POLYGON ((-62.43458 -0.77542, -62.43458 -0.776...",1.0
2,"POLYGON ((-62.46708 -0.77875, -62.46708 -0.779...",1.0
3,"POLYGON ((-62.41625 -0.77875, -62.41625 -0.779...",1.0
4,"POLYGON ((-62.48708 -0.77958, -62.48708 -0.780...",1.0
...,...,...
167952,"POLYGON ((11.12875 -3.66458, 11.12875 -3.66542...",1.0
167953,"POLYGON ((11.13042 -3.66458, 11.13042 -3.66542...",1.0
167954,"POLYGON ((11.13625 -3.73208, 11.13625 -3.73292...",1.0
167955,"POLYGON ((9.30208 -0.33375, 9.30208 -0.33875, ...",1.0


In [55]:
# Perform a spatial join to find intersections
LAU_NUTS_reproj = LAU_NUTS.to_crs("EPSG:4326")
joined_gdf = gpd.sjoin(LAU_NUTS_reproj, shapes_gdf, how='inner', predicate='intersects')

# remove columns added by shapes_gdf
joined_gdf.drop(columns=['index_right', 'value'])

# Remove duplicate rows based on the index of gdf1
LAU_NUTS_CLEANCM = joined_gdf[~joined_gdf.index.duplicated(keep='first')]
LAU_NUTS_CLEANCM

C:\Users\kras\AppData\Local\Temp\ipykernel_11464\3554839154.py:2: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None

  joined_gdf = gpd.sjoin(LAU_NUTS_reproj, shapes_gdf, how='inner', predicate='intersects')


In [70]:
#if a column contains '\' character, it will be replaced with '_' character
LAU_NUTS_CLEANCM.columns = LAU_NUTS_CLEANCM.columns.str.replace('\\', '_')

#rename columns with name FID to FID_LAU
LAU_NUTS_CLEANCM.rename(columns={'FID': 'FID_LAU'}, inplace=True)

#Create a new column that contains the index of the row
LAU_NUTS_CLEANCM['fid'] = LAU_NUTS_CLEANCM.index

C:\Users\kras\AppData\Local\Temp\ipykernel_11464\2723112034.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LAU_NUTS_CLEANCM.rename(columns={'FID': 'FID_LAU'}, inplace=True)
c:\Users\kras\AppData\Local\mambaforge\envs\coclico\Lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [71]:
#saving
LAU_NUTS_CLEANCM.to_parquet(r"p:\11207608-coclico\FULLTRACK_DATA\WP4\LAU_stats\LAU_NUTS_CFHP_CLEANCM.parquet")

# Save the GeoDataFrame to a geopackage file
LAU_NUTS_CLEAN.to_file(r'p:\11207608-coclico\FULLTRACK_DATA\WP4\LAU_stats\lau_nuts_cfhp_cleancm.gpkg', layer='lau_nuts_cfhp_cleancm', driver='GPKG')
""" gdf.to_parquet('lau_nuts_cfhp.parquet') """

" gdf.to_parquet('lau_nuts_cfhp.parquet') "

In [72]:
# HOW ABOUT THE NUTS0 and NUTS2 data?
LAU_NUTS_CLEANCM

,GISCO_ID,CNTR_CODE,nuts_2,nuts_3,LAU_ID,LAU_NAME,POP_2020,POP_DENS_2,AREA_KM2,YEAR,...,UNDEFENDED_MAPS_SLR_SSP585_2050_n_less05,UNDEFENDED_MAPS_SLR_SSP585_2050_n_more05,UNDEFENDED_MAPS_SLR_SSP585_2050_total,UNDEFENDED_MAPS_SLR_SSP585_2050_flooded,UNDEFENDED_MAPS_SLR_SSP585_2100_n_nans,UNDEFENDED_MAPS_SLR_SSP585_2100_n_less05,UNDEFENDED_MAPS_SLR_SSP585_2100_n_more05,UNDEFENDED_MAPS_SLR_SSP585_2100_total,UNDEFENDED_MAPS_SLR_SSP585_2100_flooded,fid
1192,DE_03361001,DE,DE93,DE93B,03361001,"Achim, Stadt",31923.0,471.398771,67.719735,2020,...,2.0,0.0,201630.0,0.000010,201628.0,2.0,0.0,201630.0,0.000010,1192
1193,DE_03361002,DE,DE93,DE93B,03361002,Blender,2885.0,75.034640,38.448908,2020,...,1.0,0.0,108810.0,0.000009,108809.0,1.0,0.0,108810.0,0.000009,1193
1195,DE_03361003,DE,DE93,DE93B,03361003,Dörverden,9009.0,107.940908,83.462333,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1195
1196,DE_03361004,DE,DE93,DE93B,03361004,Emtinghausen,1464.0,68.573846,21.349247,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1196
1197,DE_03361005,DE,DE93,DE93B,03361005,Kirchlinteln,9911.0,56.675823,174.871744,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97667,FR_97614,FR,FRY5,FRY50,97614,Ouangani,10203.0,555.838786,18.356042,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97667
97669,FR_97616,FR,FRY5,FRY50,97616,Sada,11156.0,1012.643086,11.016715,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97669
97670,FR_97617,FR,FRY5,FRY50,97617,Tsingoni,13934.0,406.428213,34.284037,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97670
97660,FR_97608,FR,FRY5,FRY50,97608,Dzaoudzi,17831.0,2797.974566,6.372824,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97660


In [ ]:
# Plot the results
# ax = gdf.plot(color='blue', edgecolor='black')
# shapes_gdf.plot(ax=ax, color='red', alpha=0.5)
# matching_polygons.plot(ax=ax, color='green', alpha=0.5)

# plt.show()